# 显示控制机器人
## 1.导入所需模块

In [ ]:
from nxbot import Robot,event
from IPython.display import display
import ipywidgets.widgets as widgets
from nxbot import Robot,event,bgr8_to_jpeg
import traitlets
from traitlets.config.configurable import Configurable
import ipywidgets
import numpy as np
import cv2
import time

## 2.创建摄像头RGB数据回调函数

In [ ]:
# 创建图像显示部件
image_widget = widgets.Image(format='jpeg')

# 创建接收图像数据的函数
def on_new_image(evt):
    img_data = cv2.resize(evt.dict['data'], (320,240))
    # 将图像数据传输给图像显示部件
    image_widget.value=bgr8_to_jpeg(img_data)

## 3.创建摄像头深度数据回调函数
摄像头深度信息由[640，480]的深度值矩阵构成。
1. 我们可以获取某一个点的深度值，通过滑块将这个点的深度值展示出来；
2. 我们也可以获取所有深度值，将深度值转为图像信息，通过图像的方式展示出来。

In [ ]:
# 创建滑块，将深度信息进行可视化
depth_slider = ipywidgets.FloatSlider(min=0.0, max=10000.0, description='中心点深度值')

# 创建深度图像显示窗口
depth_image_widget = widgets.Image(format='jpeg')

def on_new_depth(evt):
    
    # 获取中心点的深度值
    depth = evt.dict['data'].get_distance(320, 240)
    if depth ==0:
        depth = evt.dict['data'].get_distance(322, 241)
    depth_slider.value = depth
    # 将深度数据转换为opencv接受的格式
    depth_frame = np.asanyarray(evt.dict['data'].get_data())
    # 转换深度数据转换为图像数据
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_frame, alpha=0.03), cv2.COLORMAP_JET)
    # 改变图像大小
    show_img = cv2.resize(depth_colormap, (320,240))
    # 将图像数据传给图像显示窗口小部件
    depth_image_widget.value = bgr8_to_jpeg(show_img)

## 4.创建摄像头控制滑块

In [ ]:
# 创建摄像头视角滑块。
camera_x_slider = ipywidgets.FloatSlider(min=-90, max=90, step=1, value=0, description='摄像头左右')
camera_y_slider = ipywidgets.FloatSlider(min=-90, max=90, step=1, value=0, description='摄像头上下')

class Camera(Configurable):
    cx_speed = traitlets.Float(default_value=0.0)
    cy_speed = traitlets.Float(default_value=0.0)
    @traitlets.observe('cx_speed')
    def x_speed_value(self, change):
        time.sleep(0.1)
        self.cx_speed=change['new']
        rbt.base.set_ptz(x = self.cx_speed, y = self.cy_speed)

    @traitlets.observe('cy_speed')
    def a_speed_value(self, change):
        time.sleep(0.1)
        self.cy_speed=change['new']
        rbt.base.set_ptz(x = self.cx_speed, y = self.cy_speed)

camera = Camera()

camera_x_link = traitlets.dlink((camera_x_slider,'value'), (camera, 'cx_speed'), transform=lambda x: x)
camera_y_link = traitlets.dlink((camera_y_slider,'value'), (camera, 'cy_speed'), transform=lambda x: x)

## 5.图形化界面操控机器人
通过“ipywidgets”工具包下面的“widgets”模块可以让我们通过图形界面来操控小车。
下面我们先创建一个滑块，然后通过“IPython”工具包的“display”模块将滑块显示出来。
1. 导入所需工具包；
2. 创建前后左右，停止，左平移，右平移按钮控制按钮；
3. 设置默认运行速度和时间；
4. 将所有运动模式分别对接到对应的控制按钮上面，并通过“on_click”方法来激活小车进行运动，“on_click”表示鼠标左键单击。
5. 将按键显示在窗口
> 注意：此时点击按钮将会触发机器人进行运动，请让机器人周围保持空旷，同时注意安全，避免被小车撞到。

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

# 创建按钮外观。
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')

#创建控制按钮。
stop_button = widgets.Button(description='停止', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='前进', layout=button_layout)
backward_button = widgets.Button(description='后退', layout=button_layout)
left_button = widgets.Button(description='左转', layout=button_layout)
right_button = widgets.Button(description='右转', layout=button_layout)
shiftleft_button = widgets.Button(description='左平移', layout=button_layout)
shiftright_button = widgets.Button(description='右平移', layout=button_layout)

# 默认运行速度和时间。
speed = 0.3
limit_time = 2

#定义所有运动模式。
def stop(change):
    rbt.base.stop()

def step_forward(change):
    rbt.base.forward(speed, limit_time)

def step_backward(change):
    rbt.base.backward(speed, limit_time)

def step_left(change):
    rbt.base.turnleft(speed, limit_time)

def step_right(change):
    rbt.base.turnright(speed, limit_time)
    
def shift_left(change):
    rbt.base.shiftleft(speed, limit_time)

def shift_right(change):
    rbt.base.shiftright(speed, limit_time)
    
# 通过“on_click”方法来触发小车进行运动。
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
shiftleft_button.on_click(shift_left)
shiftright_button.on_click(shift_right)

# 实例化机器人对象
rbt = Robot()
# 把按键拼接在一起。
if rbt.name=='dachbot':
    up_box = widgets.HBox([shiftleft_button, forward_button, shiftright_button], layout=widgets.Layout(align_self='center'))

elif rbt.name=='dbot':
    up_box = widgets.HBox([forward_button], layout=widgets.Layout(align_self='center'))
        
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([up_box, middle_box, backward_button])

# 显示控制操作界面。

## 6.连接机器人，显示所有操作小部件

In [ ]:
rbt.connect()

if rbt.name=='dachbot':
    # 打开摄像头
    rbt.camera.start(enable_depth_stream=True)
    # 监听摄像头深度数据，并将数据传给“on_new_depth”函数
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_DEPTH,on_new_depth)
    # 监听摄像头图像数据，并将数据传给“on_new_image”函数
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
    # 通过"display"模块将深度值显示在滑块上。
    slider_box = ipywidgets.VBox([depth_slider, camera_y_slider])    
    
    image_box = ipywidgets.HBox([image_widget,depth_image_widget])

    display(image_box)
    
elif rbt.name=='dbot':
     # 打开摄像头
    rbt.camera.start()
    # 监听摄像头图像数据，并将数据传给“on_new_image”函数
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
    
    slider_box = ipywidgets.VBox([camera_x_slider, camera_y_slider])    
    # 显示图像
    display(image_widget)

# 显示控制滑块
display(widgets.HBox([controls_box, slider_box]))
    

## 7.断开与机器人连接

In [ ]:
# rbt.disconnect()